In [1]:
import sys  
reload(sys)  
sys.setdefaultencoding('utf-8')

In [2]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import tldextract
from collections import Counter


In [3]:
import pandas as pd
import json

In [4]:
from sklearn.externals.six.moves import html_parser
from sklearn.externals.six.moves import urllib
from sklearn.datasets import get_data_home
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF
from sklearn.naive_bayes import MultinomialNB

In [5]:
from sklearn.datasets import make_blobs
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.cross_validation import train_test_split

In [7]:
#load customized stop words
with open('/Users/nhu2000/projects/SimilarSearch/data/stop_words/Baidu_Stop_Words.json') as data_file:    
    data = json.load(data_file)
bd_stop_words = list(data)

In [8]:
#vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5, stop_words='english')
vectorizer = TfidfVectorizer(analyzer = 'word', stop_words = bd_stop_words)

In [8]:
#Latest Random forest classifier
#dataset size = 35095  score = 0.960982747058
#dataset size = 44709  score = 0.977663418373 (0.9951 after tuning)
#dataset size = 83217  score = 0.9949 

In [9]:
clf = RandomForestClassifier(n_estimators=50)  #0.960982747058
#clf = RidgeClassifier(tol=1e-2, solver="lsqr")  #0.936901786259
#clf = Perceptron(n_iter=500)   #0.931597878437
#clf = PassiveAggressiveClassifier(n_iter=500)   #0.952935438639
#clf = KNeighborsClassifier(n_neighbors=200) #0.895506919466

In [82]:
#df_no_dup = pd.read_csv('/Users/nhu2000/projects/SimilarSearch/data/segments/title-jiaba-segment_v3.csv')
df_no_dup = pd.read_csv('/Users/nhu2000/projects/SimilarSearch/data/classifier/huizhou_v1.csv')

In [83]:
df_no_dup.info()

In [84]:
df_no_dup['target'].sum()

17318

df_ref = pd.read_csv('../data/segments/solr-more/solr-series-segment.csv')

df_ref.head()

len(df_no_dup[df_no_dup['flag'] == 'pos'])

In [85]:
#fine tune for Stanford segmenter
#df_no_dup['token title'] = df_no_dup['token title'].map(lambda x: x.replace(u'徽 文化', u'徽文化') if u'徽 文化' in x else x)
#df_no_dup['segment'] = df_no_dup['segment'].map(lambda x: x.replace(u'徽 文化', u'徽文化'))
#df_no_dup['segment'] = df_no_dup['segment'].map(lambda x: x.replace(u'胡 适', u'胡适'))
#df_no_dup['segment'] = df_no_dup['segment'].map(lambda x: x.replace(u'徽 茶', u'徽茶'))


In [86]:
X_articles = df_no_dup['segment'].values
X = vectorizer.fit_transform(X_articles)

In [87]:
Y = df_no_dup['target'].values

X_train, X_test, Y_train, Y_test = train_test_split(X, Y,
    test_size=0.8, random_state=0)

X_train = X
Y_train = Y

test = pd.read_csv('../data/segments/titles/xinhs_huixueyuandi-all.csv')
test['flag'] = 1
test['segments'] = test.segments.map(lambda x: x.replace(u'徽 文化', u'徽文化'))
test_articles = test['segments'].values

X_test = vectorizer.transform(test_articles)
Y_test = test['target']

In [88]:
# mask for train and test
msk = np.random.rand(len(df_no_dup)) < 0.8
train = df_no_dup[msk]
test = df_no_dup[~msk]

In [21]:
# use main data for train and incoming for test
train = df_no_dup
#test = pd.read_csv('../data/segments/news/google-huizhou-news_segments.csv')
#test = pd.read_csv('../data/segments/news/baidu-huizhou-news_segments.csv')|
test = pd.read_csv('/Users/nhu2000/projects/SimilarSearch/data/clean/huizhou_full_v1_1.csv')
#test = pd.read_csv('/Users/nhu2000/Desktop/huizhou/data/huiwenhua/solr_2/Solr-huizhou-final-title-jieba-segment.csv', encoding='utf-8')


In [22]:
test.info()

In [28]:
test.head()

,url,title,summary,s_vec_cnt,content,content_seg
0,http://cul.anhuinews.com/system/2016/11/28/007...,洪建华夫妇的竹刻人生,。不久前，我们几名政协委员应邀到正在建设中的黄山市徽派雕刻博物馆参观考察时，洪建华一边介绍新...,"{u'融入': 1, u'国内外': 1, u'夫妻俩': 1, u'艺术馆': 2, u'...",谁能想到，洪建华夫妇这对一无学历、二无家传、三无师承的“三无大师”成了省级非物质文化遗产·徽...,谁 能 想到 ， 洪 建华 夫妇 这 对 一 无 学历 、 二 无 家传 、 三无 师承 的...
1,http://cul.anhuinews.com/system/2016/11/21/007...,徽州民歌传人操明花校园授艺,。“磨米磨麦，磨米磨麦……”随着教室里传来一阵阵悦耳动听的方言民歌声，操明花正兴致勃勃地教同...,"{u'传来': 1, u'小学': 1, u'原汁原味': 1, u'明花': 1, u'磨...",近日，省非物质文化遗产徽州民歌传承人操明花前往休宁县海阳二小、歙县城关小学和徽州区岩寺镇中心...,近日 ， 省非 物质 文化遗产 徽州 民歌 传承 人操 明花 前往 休宁县 海阳 二小 、 ...
2,http://cul.anhuinews.com/system/2016/11/03/007...,黄山楹联漫谈,。黄山雄伟奇丽、气势非凡，原省文联主席赖少其说，非有大手笔，不能画黄山。有“二联”站得高，看...,"{u'气派': 1, u'名句': 1, u'唱道': 1, u'千丈': 1, u'凿刻'...",黄山楹联第一人当推明朝诗人余绍祉。他遍历祖国名山大川，见到无数奇峰怪石，却感到抵不上黄山峰林...,黄山 楹联 第一 人当 推 明朝 诗人 余绍祉 。 他 遍历 祖国 名山大川 ， 见到 无数...
3,http://cul.anhuinews.com/system/2016/01/26/007...,徽派雕刻——徽州木雕艺术,徽州民居、家具木板和圆木雕刻艺术的简称。木雕在旧属徽州各县分布之广在全国屈指可数，宅院内的屏...,"{u'吉祥图案': 1, u'少数': 1, u'显示': 1, u'内': 1, u'木板...",徽州民居、家具木板和圆木雕刻艺术的简称。木雕在旧属徽州各县分布之广在全国屈指可数，宅院内的屏...,徽州 民居 、 家具 木板 和 圆木 雕刻 艺术 的 简称 。 木雕 在 旧属 徽州 各县 ...
4,http://cul.anhuinews.com/system/2016/01/26/007...,徽派雕刻——徽州砖雕艺术,。从早期的简单粗犷、朴素的纹样，逐步演变到后期的繁腹、细致华丽的结构，形成了自己独特的艺术风...,"{u'嵌窃时': 1, u'要求': 1, u'厅': 1, u'严谨': 1, u'颇':...",徽州砖雕徽州砖雕的造型，来源于汉画象砖。汉画象砖大都是模印砖坯刻画后入密烧制，嵌窃时再进行修...,徽州 砖雕 徽州 砖雕 的 造型 ， 来源于 汉 画象 砖 。 汉 画象 砖 大都 是 模印...


In [54]:
global num
num = 0
def map_unserialize (x, eval_val = False):
    global num
    #print 'incoming'
    #print x
    #print '----- in'
    #print x
    d = dict()
    s = x.strip('{}')
    #s = s.strip('[]')
    ret = s.split(',')
    
    len(ret)
    
    #print len(ret)
    #print ret
    idx = 0
    for t in ret:
        #print idx, t
        if  idx < 100:
            r = t.split(':')
            #print r[0], r[1]
            key = eval(r[0].decode('utf-8'))
            d[key] = float(r[1])
            idx += 1
#        else:
#            print 'found the end'
    #print len(ret)
    #print 'split -----'
    #print ' '.join(ret)
    print num
    num += 1
    return d

In [55]:
from time import time
t0 = time()
test['s_vec'] = test.s_vec_cnt.map(lambda x: map_unserialize(x))
print time() - t0 

In [61]:
test['segment'] = test.s_vec.map(lambda x: ' '.join(x.keys()))

In [66]:
test['target'] = 0

In [63]:
print (test.iloc[1]['segment'])

In [64]:
mask = test['segment'].str.len() > 0
test = test[mask]

In [65]:
test.info()

train['token title'] = train['token title'].map(lambda x: x.replace(u'徽 文化', u'徽文化'))
train['token title'] = train['token title'].map(lambda x: x.replace(u'胡 适', u'胡适'))
train['token title'] = train['token title'].map(lambda x: x.replace(u'徽 茶', u'徽茶'))

#test['segments'] = test.segments.map(lambda x: x.replace(u'徽 文化', u'徽文化') if u'徽 文化' in x else x)
test['token title'] = test['token title'].map(lambda x: x.replace(u'徽 文化', u'徽文化'))
test['token title'] = test['token title'].map(lambda x: x.replace(u'胡 适', u'胡适'))
test['token title'] = test['token title'].map(lambda x: x.replace(u'徽 茶', u'徽茶'))


train['token title'] = train['token title'].map(lambda x: x.replace(u'徽 文化', u'徽文化') if u'徽 文化' in x else x)

for t in train['token title']:
    print t

if u'徽 文化' in u'唤醒 徽 文化 “ 活化石 ”':
    t = '唤醒 徽 文化 “ 活化石 ”'
    t1 = t.replace('徽 文化', '徽文化')
    print t1

In [89]:
Y_train = train['target']
Y_test = test['target']

print len(Y_train)

train.head(100)

test.head()

print len(train), len(test)

In [90]:
train_articles = train['segment'].values

In [91]:
#test_articles = test['combined'].values
test_articles = test['segment'].values
#test_articles = test['segments'].values
test_url = test['url'].values
test_flag = test['target'].values

print len(train_articles), len(test_articles)

In [92]:
X_train = vectorizer.fit_transform(train_articles)

In [93]:
clf = RandomForestClassifier(n_estimators=50)  #0.960982747058 -- 0.99586 the best

In [94]:
clf.fit(X_train, Y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=50, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [95]:
X_test = vectorizer.transform(test_articles)

vectorizer = TfidfVectorizer(analyzer = 'word', stop_words = Baidu_CH_Stopwords)
document_term_mat = vectorizer.fit_transform(train_articles)
words = vectorizer.get_feature_names()

In [101]:
Y_pred = clf.predict(X_test)

In [102]:
scores = metrics.accuracy_score(Y_test, Y_pred)

In [103]:
print sum(Y_test), sum(Y_pred)

In [79]:
test['target'] = Y_pred
check = test[test['target'] == 1]
check.info()
check.head()

,url,title,summary,s_vec_cnt,content,content_seg,s_vec,segment,target
3,http://cul.anhuinews.com/system/2016/01/26/007...,徽派雕刻——徽州木雕艺术,徽州民居、家具木板和圆木雕刻艺术的简称。木雕在旧属徽州各县分布之广在全国屈指可数，宅院内的屏...,"{u'吉祥图案': 1, u'少数': 1, u'显示': 1, u'内': 1, u'木板...",徽州民居、家具木板和圆木雕刻艺术的简称。木雕在旧属徽州各县分布之广在全国屈指可数，宅院内的屏...,徽州 民居 、 家具 木板 和 圆木 雕刻 艺术 的 简称 。 木雕 在 旧属 徽州 各县 ...,"{u'吉祥图案': 1.0, u'少数': 1.0, u'显示': 1.0, u'内': 1...",吉祥图案 少数 显示 内 木板 雕刻 杉树 选材 楠木 床 为贵 用具 情调 梓 分布 花鸟...,1
5,http://cul.anhuinews.com/system/2016/01/26/007...,徽派雕刻——徽州石雕艺术,。徽州石雕题材受雕刻材料本身限制，不及木雕与砖雕复杂，主要是动植物形象、博古纹样和书法，至于...,"{u'从嶙刚': 1, u'内': 1, u'旁有': 1, u'牌坊': 3, u'左为'...",传统“徽州四雕”之一。在徽州城乡布很广，类别亦多，主要用于寺宅的廊柱、门墙、牌坊、墓葬等处的...,传统 “ 徽州 四雕 ” 之一 。 在 徽州 城乡 布 很 广 ， 类别 亦 多 ， 主要 ...,"{u'内': 1.0, u'旁有': 1.0, u'天井': 1.0, u'左为': 1.0...",内 旁有 天井 左为 壁上 融 意象 怪石 甚高 砖雕 工艺 宅 可谓 浮雕 小草 溪畔 色...,1
6,http://cul.anhuinews.com/system/2016/01/26/007...,徽州古民居雕刻装饰研究,。第一，徽州民居雕刻装饰中的任何形态，其来源都不是为美而美的。一是融装饰艺术与建筑结构于一体...,"{u'言': 1, u'自然': 1, u'三绝': 1, u'三回': 1, u'处理':...",徽州因明清两朝的繁盛而在历史上留下浓墨重彩的一笔，建筑就是那个时代的重要表现之一。当时的徽商...,徽州 因 明清 两朝 的 繁盛 而 在历史上 留下 浓墨重彩 的 一笔 ， 建筑 就是 那个...,"{u'言': 1.0, u'三绝': 1.0, u'三回': 1.0, u'处理': 1.0...",言 三绝 三回 处理 合 自然 通过 窗上 成 情感 高墙 砖雕 归结为 四友 海峡两岸 及...,1
7,http://cul.anhuinews.com/system/2016/01/26/007...,徽派雕刻——忆千年沧桑，探秘西递牌楼,。西递村呈船形，村中鳞次栉比的古民居建筑群，就像一间间船舱，组成大船的船体；昔日村头高大的乔...,"{u'融入': 1, u'退': 1, u'内': 3, u'首先': 1, u'牌坊': ...",西递位于安徽省黄山市黟县城东8公里处，因地处古徽州府之西，曾设驿站“铺递所”而得名。作为唐李...,西递 位于 安徽省 黄山市 黟县 城东 8 公里 处 ， 因 地处 古 徽州 府 之西 ， ...,"{u'融入': 1.0, u'退': 1.0, u'内': 3.0, u'首先': 1.0,...",融入 退 内 首先 牌坊 船形 堪称 整体 旌表 蒙馆 通连 之地 清咸丰 本邑 西侧 厅 ...,1
12,http://cul.anhuinews.com/system/2014/07/14/006...,徽派雕刻——古老徽州明清建筑中木雕艺术,。即便是现在，当您到徽州来旅行，一踏进这座山国地界，给您的第一印象是：在那碧玉簪般的丛山之间...,"{u'遐想': 1, u'形式': 1, u'三雕': 1, u'砌': 2, u'庙宇':...",近年来国内及欧美、日本、东南亚等地建筑学界的学者和其他方面的专家来徽州考察三雕艺术与古建筑者...,近年来 国内 及 欧美 、 日本 、 东南亚 等 地 建筑 学界 的 学者 和 其他 方面 ...,"{u'遐想': 1.0, u'瞪': 1.0, u'三雕': 1.0, u'砌': 2.0,...",遐想 瞪 三雕 砌 庙宇 栏板 碧玉簪 祠堂 花台 上方 栏杆 徽州 然而 国内 其他 民居...,1


In [104]:
print scores

In [81]:
test.to_csv('/Users/nhu2000/projects/SimilarSearch/data/classifier/huizhou_v1.csv', encoding='utf-8', index=False)

In [449]:
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC

In [450]:
lr = LogisticRegression()
gnb = GaussianNB()
svc = LinearSVC(C=1.0)
rfc = RandomForestClassifier(n_estimators=50)

In [451]:
lr = LogisticRegression(max_iter=1000)

In [452]:
lr.fit(X_train, Y_train)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=1000, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [453]:
Y_pred = lr.predict(X_test)
scores = metrics.accuracy_score(Y_test, Y_pred)
print scores

0.967582725743


In [37]:
gnb = GaussianNB()
gnb.fit()

In [50]:

svc = LinearSVC(C=1.0, max_iter=5000)

In [57]:
svc.fit(X_train, Y_train)


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=5000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [33]:
Y_pred = svc.predict(X_test)
scores = metrics.accuracy_score(Y_test, Y_pred)
print scores

NameError: name 'svc' is not defined

In [35]:
my_pickle_dump("../pkl/RandomForest.pkl", clf)

In [233]:
print sum(Y_test), sum(Y_pred)

if u'徽 文化' in u'唤醒 徽 文化 “ 活化石 ”':
    t = '唤醒 徽 文化 “ 活化石 ”'
    t1 = t.replace('徽 文化', '徽文化')
    print t1

In [105]:
cnt = 0
for i in range(0, len(Y_pred)):
    if (Y_test.values[i] != Y_pred[i]):
    #if (Y_pred[i] == 1):
        #if ('' in test_articles[i]):
            #print test_articles[i]
        print i, cnt, Y_test.values[i], Y_pred[i]
        print test_articles[i]
        cnt += 1

In [235]:
print cnt

In [100]:

cnt = 0
for i in range(0, len(Y_pred)):
    if (Y_test.values[i] != Y_pred[i]):
    #if (Y_pred[i] == 0):
        if i == 34 or i == 212 or i == 348 or i == 349 or i == 608 or i == 1389 or i == 1418:
            Y_pred[i] = 1 
        elif i == 130 or i == 133: 
            Y_pred[i] = 0 
            #Y_test.values[i] = 1
            print i, cnt, Y_test.values[i], Y_pred[i]
            print test_articles[i]


In [187]:
print cnt

In [273]:
def get_columns3():
    return ('url', 'domain', 'title', 'segment', 'key_words', 'summary', 'body', 'target') 
def get_empty_columns3():
    return ('', '', '', '', '', '', '', '') 

In [346]:
idx = 0
new_df = pd.DataFrame(columns = get_columns3())

In [350]:
new_df['url'] = test['url']
new_df['domain'] = test['domain']
new_df['title'] = test['title']
new_df['segment'] = test['segment']
new_df['key_words'] = test['key_words']
new_df['summary'] = test['summary']
new_df['combined'] = test['combined']
new_df['target'] = Y_pred
new_df['body'] = test['body']

In [351]:
new_df.to_csv('/Users/nhu2000/Desktop/huizhou/data/huiwenhua/huizhou_web_contentv5.csv', encoding='utf-8', index=False)

In [254]:
new_df.info()

In [252]:
sum(Y_pred), sum(new_df['target'])

(7880, 7880)

In [503]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9019 entries, 3 to 44513
Data columns (total 6 columns):
url        9019 non-null object
domain     9019 non-null object
title      9019 non-null object
segment    9019 non-null object
target     9019 non-null int64
source     9019 non-null object
dtypes: int64(1), object(5)
memory usage: 493.2+ KB


In [151]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35639 entries, 1 to 44513
Data columns (total 6 columns):
domain     35639 non-null object
segment    35639 non-null object
source     0 non-null float64
target     35639 non-null int64
title      35639 non-null object
url        35639 non-null object
dtypes: float64(1), int64(1), object(4)
memory usage: 1.9+ MB


In [152]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8875 entries, 0 to 44507
Data columns (total 6 columns):
domain     8875 non-null object
segment    8875 non-null object
source     0 non-null float64
target     8875 non-null int64
title      8875 non-null object
url        8875 non-null object
dtypes: float64(1), int64(1), object(4)
memory usage: 485.4+ KB


In [153]:
new_df.head(200)

,url,domain,title,segment,target,source
0,http://www.hzwh.com/article.asp?id=2533,hzwh.com,推动徽州传统文化的普及宣传,推动 徽州 传统 文化 的 普及 宣传,1,
7,www.hzwh.com/article.asp?id=1156,hzwh.com,名山文化与泰山文化,名山 文化 与 泰山 文化,0,
11,www.hzwh.com/article.asp?id=996,hzwh.com,为什么说“十姓九汪”,什么 为什么 说 “ 十姓 九 汪 ”,0,
12,www.hzwh.com/article.asp?id=992,hzwh.com,“红顶商人”指谁,“ 红顶 商人 红顶商人 ” 指 谁,0,
16,www.hzwh.com/article.asp?id=895,hzwh.com,徽州宗祠与宗祠文化,徽州 宗祠 与 宗祠 文化,1,
23,www.hzwh.com/article.asp?id=715,hzwh.com,朱熹与植树,朱熹 与 植树,1,
28,www.hzwh.com/article.asp?id=986,hzwh.com,走南闯北的贩运贸易,走南闯北 的 贩运 贸易,0,
29,www.hzwh.com/article.asp?id=902,hzwh.com,岩寺“上九”庙会,岩寺 “ 上 九 ” 庙会,0,
30,www.hzwh.com/article.asp?id=830,hzwh.com,明清以来徽州典当业在城市的分布和发展,明清 以来 徽州 典当 典当业 在 城市 的 分布 和 发展,1,
35,www.hzwh.com/article.asp?id=984,hzwh.com,徽商的赊销赊购经营,徽商 的 赊销 赊购 经营,1,


In [207]:
joined = train.append(new_df)

In [208]:
joined.info()

final = joined[['url', 'domain', '', '', flag']]

In [609]:
joined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44611 entries, 0 to 44511
Data columns (total 6 columns):
domain     44611 non-null object
segment    44611 non-null object
source     8957 non-null object
target     44611 non-null int64
title      44611 non-null object
url        44611 non-null object
dtypes: int64(1), object(5)
memory usage: 2.4+ MB


In [209]:
df_no_dup = joined.dropna(subset =['url']) 

In [210]:

df_no_dup = df_no_dup.dropna(subset =['segment']) 

In [211]:

df_no_dup = df_no_dup.drop_duplicates(['url'])


In [79]:

df_no_dup = df_no_dup.drop_duplicates(['segment'])

In [212]:
df_no_dup.info()

In [213]:
df_no_dup['target'].sum()

11904

In [214]:
df_no_dup.to_csv('/Users/nhu2000/Desktop/huizhou/data/huiwenhua/classifier/title-jiaba-segment_v3_0.csv', encoding='utf-8', index=False)

In [51]:
print cnt, cnt*1.0/len(Y_pred)

In [215]:
print df_no_dup['flag'].sum()

KeyError: 'flag'

In [ ]:
#Grid search

In [ ]:
from sklearn.datasets import make_classification
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, \
    matthews_corrcoef, make_scorer
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import train_test_split

In [ ]:
def overall_average_score(actual,prediction):
    precision, recall, f1_score, _ = precision_recall_fscore_support(
        actual, prediction, average='binary')
    total_score = (matthews_corrcoef(actual, prediction) +
        accuracy_score(actual, prediction) + precision + recall + f1_score)
    return total_score / 5

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
    test_size=0.5, random_state=0)

In [ ]:
grid_scorer = make_scorer(overall_average_score, greater_is_better=True)

In [ ]:
parameters = {'n_estimators': [10,20,30],
              'max_features': ['auto','sqrt','log2',0.5,0.3], }
gs_rf = GridSearchCV(clf, parameters, cv=5, scoring=grid_scorer)
gs_rf.fit(X_train,y_train)